In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import wcs
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn
from pathlib import Path
from math import trunc
import copy
%matplotlib inline


SMALL_SIZE = 10*2                                        
MEDIUM_SIZE = 12*2
BIGGER_SIZE = 14*2

plt.rc('font', size=SMALL_SIZE, family='serif')          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)                     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)                    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE, direction='in')    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE, direction='in')    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE/1.5)                    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)                  # fontsize of the figure title
plt.rc('figure', figsize='8, 6')                         # size of the figure, used to be '4, 3' in inches

In [ ]:
# read in filterdata
d = ascii.read('filter_transmissions/WMN_ASASSN-LCOGT.txt')

filt, wmean  = np.array(d['FILT']),np.array(d['WMEAN'])
print (filt,wmean)

wmean_dict = {}
for i in range(len(filt)):
    plt.plot([wmean[i]]*2,[0,1],label=filt[i])
    wmean_dict[filt[i]] = wmean[i]
print (wmean_dict)

plt.legend()
plt.xlabel('Wavelength [nm]')
plt.title("Filter transmission weighted mean")
plt.show()


In [ ]:
# Read in J0600 data
J0600_dat = ascii.read('J0600_all.ecsv')
obs_mask = J0600_dat['Observer Code'] == 'HMB'


filters = np.array(J0600_dat['Band'][obs_mask])
mag = np.array(J0600_dat['Magnitude'][obs_mask])
epoch = np.array(J0600_dat['MJD'][obs_mask])
Nrow = len(J0600_dat['JD'][obs_mask])

print (J0600_dat[0])

In [ ]:
# Make a mask for all the filters:
J0600_filt = [] #list of the different filters
NimgINf = {} #dictionary of number of images for each filter
for f in filters:
    if f not in J0600_filt:
        J0600_filt.append(f)
        NimgINf[f] = 1
    else:
        NimgINf[f] +=1
print (filt)

fmask = {}
for f in J0600_filt:
    fmask[f] = filters == f

plt.figure(figsize=(16,8))
for f in J0600_filt:
    plt.errorbar(epoch[fmask[f]],mag[fmask[f]],fmt='.',label=f)
plt.legend()
plt.gca().invert_yaxis()
plt.title('HMB magnitude for J0600')
plt.ylabel("Magnitude")
plt.xlabel("Epoch [MJD]")
plt.show()

In [ ]:
#Last blob of LCOGT data between 59273 & 59280 MJD

plt.figure(figsize=(6,4.5))
for e in [59273,59280,59355,59365]:
    plt.plot([e]*2,[15.5,12.1],c='black',linestyle='--')

for f in J0600_filt:
    plt.errorbar(epoch[fmask[f]],mag[fmask[f]],fmt='.',label=f)
plt.legend(loc=2)
plt.gca().invert_yaxis()
plt.title('ASAS-SN magnitude for J0600')
plt.ylabel("Magnitude")
plt.xlabel("Epoch [MJD]")
plt.axis(xmin=59250,xmax=59368,ymin=15.5,ymax=12)
plt.show()

HMB_dmagdict = {}
for f in ['V','I','B','R']:
    mask1 = (epoch[fmask[f]]>59273) * (epoch[fmask[f]]<59280)
    mag1 = np.median(mag[fmask[f]][mask1])
    mask2 = epoch[fmask[f]] > 59355
    mag2 = np.mean(mag[fmask[f]][mask2])
    #print (mag2)
    #print (np.mean(mag[fmask[f]][epoch[fmask[f]] > 59355]))
    HMB_dmagdict[f] = mag1-mag2
print (HMB_dmagdict)
print (epoch[fmask['B']][-1])


for f in J0600_filt:
    plt.scatter(wmean_dict[f],HMB_dmagdict[f],label='HMB '+f,zorder=1)
plt.legend()
plt.axis(ymin=0,xmin=400,xmax=900)
plt.show()

In [ ]:
# Go from dicts to lists
HMB_dmag = []
HMB_wmean = []
for f in ['B','V','R','I']:
    HMB_dmag.append(HMB_dmagdict[f])
    HMB_wmean.append(wmean_dict[f])
HMB_dmag = np.array(HMB_dmag)
HMB_wmean = np.array(HMB_wmean)

# Fit line through filter depths
from scipy import optimize
def linear(x,a,b):
    return (a*x) + b
def exponent(x,A,alph,x0,B):
    return A*np.exp(-alph*(x-x0))+B

popt1, pcov1 = optimize.curve_fit(linear, HMB_wmean[:-1],HMB_dmag[:-1])
popt2, pcov2 = optimize.curve_fit(exponent, HMB_wmean,HMB_dmag,p0=[0.125,0.1,650,0.1])
print (popt1)
print (popt2)

In [ ]:
# Plot depth of curve against filter wmean
filt_c = {'gp':'g','rp':'r','ip':'indianred'}
x = np.linspace(300,900,100)
plt.figure(figsize=(5,4))
#plt.plot(x,linear(x,*popt1),c='black',linestyle='-',zorder=0)
plt.plot(x,exponent(x,*popt2),c='black',linestyle='--',zorder=0,label='Exp fit',linewidth=2)
for f in ['gp','rp','ip']:
    plt.plot([wmean_dict[f]]*2,[0,1],c=filt_c[f],zorder=1,linewidth=5,label="LCOGT "+f)
for f in J0600_filt:
    plt.scatter(wmean_dict[f],HMB_dmagdict[f],label='ASAS-SN '+f,zorder=1,s=80)



LCOGT_wmean = []
for f in ['gp','rp','ip']:
    LCOGT_wmean.append(wmean_dict[f])
LCOGT_wmean = np.array(LCOGT_wmean)

#LCOGT_dmag = linear(LCOGT_wmean,*popt1)
LCOGT_dmag = exponent(LCOGT_wmean,*popt2)

plt.legend(bbox_to_anchor=(1, 1))
plt.axis(ymin=0,xmin=380,xmax=900,ymax=.21)
#plt.scatter(LCOGT_wmean,LCOGT_dmag,zorder=5,c='r')
plt.xlabel("Wavelength $[nm]$")
plt.ylabel("$\Delta$mag")
plt.title("ASAS-SN magnitude dip against wavelength")
plt.show()

In [ ]:
print (LCOGT_dmag)
fname = 'LCOGT_ZP.txt'
fcont = Table([['gp','rp','ip'],LCOGT_dmag],names=['FILT','DMAG'])
ascii.write(fcont,fname,overwrite=True)